In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from tensorflow.keras.utils import to_categorical

data = pd.read_csv('processed_data/with_ranges_features.csv')
data['Time'] = pd.to_datetime(data['Time'], utc=True)
pd.set_option('display.max_rows', 500)

In [3]:
nr_values = 128
features = ['Elevation', 'Distance', 'HeartRateQuotient', 'HeartRateRange', 'Cadence', 'Speed', 'SpeedRange', 'SpeedQuotient', 'HeartRateClass']

In [4]:
runs = list(data.groupby("RunID"))
X = np.zeros((len(runs), nr_values, len(features)))
y = np.zeros((len(runs),))
for i, (name, group) in enumerate(runs):
    x_part = group[features].dropna()
    if len(x_part) < nr_values:
        continue

    last_30_seconds_data = group[group['Time'] >= group['Time'].max() - pd.Timedelta(seconds=30)]

    x_part["Distance"] = last_30_seconds_data['Distance'].max()

    X[i, :nr_values, :] = x_part.head(nr_values).to_numpy()
    y[i] = last_30_seconds_data['HeartRateClass'].mode().iloc[0]

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
num_classes = len(label_encoder.classes_)
y = to_categorical(y, num_classes=num_classes)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
def fit_model(model):
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f'./logs_{model.name}/run', histogram_freq=1)
    modelckpt_callback = keras.callbacks.ModelCheckpoint(
        monitor="val_loss",
        filepath=f'{model.name}_checkpoint.weights.h5',
        verbose=1,
        save_weights_only=True,
        save_best_only=True,
    )

    # Train the model
    return model.fit(
        x_train, y_train,
        validation_data=(x_test, y_test),
        epochs=50,
        callbacks=[tensorboard_callback, modelckpt_callback],
        batch_size = 10
    )

In [6]:
def build_lstm():
    model = keras.Sequential([
        keras.layers.LSTM(5, input_shape=(nr_values, len(features))),
        keras.layers.Dense(num_classes, activation="softmax"),
    ], name = "LSTM")
    model.summary()
    model.compile(optimizer='adam', metrics=["accuracy"], loss="categorical_crossentropy")
    return model

In [7]:
lstm_checkpoints = fit_model(build_lstm())

Model: "LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5)                 300       
                                                                 
 dense (Dense)               (None, 7)                 42        
                                                                 
Total params: 342
Trainable params: 342
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50


2024-06-23 10:40:50.199641: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-23 10:40:50.200665: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-23 10:40:50.201407: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

36/39 [==========================>...] - ETA: 0s - loss: 1.9252 - accuracy: 0.2694
Epoch 1: val_loss improved from inf to 1.88354, saving model to LSTM_checkpoint.weights.h5


2024-06-23 10:40:52.200102: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-23 10:40:52.201347: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-23 10:40:52.202449: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

39/39 [==============================] - 2s 27ms/step - loss: 1.9201 - accuracy: 0.2701 - val_loss: 1.8835 - val_accuracy: 0.2680
Epoch 2/50
36/39 [==========================>...] - ETA: 0s - loss: 1.8574 - accuracy: 0.2722
Epoch 2: val_loss improved from 1.88354 to 1.82539, saving model to LSTM_checkpoint.weights.h5
39/39 [==============================] - 1s 18ms/step - loss: 1.8567 - accuracy: 0.2701 - val_loss: 1.8254 - val_accuracy: 0.2680
Epoch 3/50
37/39 [===========================>..] - ETA: 0s - loss: 1.8053 - accuracy: 0.2676
Epoch 3: val_loss improved from 1.82539 to 1.77493, saving model to LSTM_checkpoint.weights.h5
39/39 [==============================] - 1s 19ms/step - loss: 1.8019 - accuracy: 0.2701 - val_loss: 1.7749 - val_accuracy: 0.2680
Epoch 4/50
36/39 [==========================>...] - ETA: 0s - loss: 1.7619 - accuracy: 0.2750
Epoch 4: val_loss improved from 1.77493 to 1.72822, saving model to LSTM_checkpoint.weights.h5
39/39 [==============================] - 1s

In [102]:
from tcn import TCN

def build_tcn():
    model = keras.Sequential([
        keras.layers.Input(shape=(nr_values, len(features))),
        TCN(
            nb_filters=5,
            kernel_size=3,
            dilations=[2**i for i in range(5)],
            nb_stacks=1,
        ),
        keras.layers.Dense(num_classes, activation='softmax')
    ], name="TCN")

    model.summary()
    model.compile(optimizer='adam', metrics=["accuracy"], loss="categorical_crossentropy")
    return model

In [103]:
tcn_checkpoints = fit_model(build_tcn())

Model: "TCN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn_3 (TCN)                 (None, 5)                 910       
                                                                 
 dense_15 (Dense)            (None, 7)                 42        
                                                                 
Total params: 952
Trainable params: 952
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
28/39 [====================>.........] - ETA: 0s - loss: 2230.5715 - accuracy: 0.0179    
Epoch 1: val_loss improved from inf to 889.24493, saving model to TCN_checkpoint.weights.h5
39/39 [==============================] - 2s 13ms/step - loss: 1871.1968 - accuracy: 0.0234 - val_loss: 889.2449 - val_accuracy: 0.0206
Epoch 2/50
30/39 [======================>.......] - ETA: 0s - loss: 530.4242 - accuracy: 0.0333  
Epoch 2: val_loss improved from 889